In [1]:
import pandas as pd
import re

**LOADING DATA FROM CSV**

In [2]:
pd.set_option('display.max_columns', None)
shark_attacks = pd.read_csv("../data/attacks.csv", encoding="latin1")
shark_attacks.columns = [i.lower().replace(' ', '_') for i in shark_attacks.columns]
shark_attacks

,case_number,date,year,type,country,area,location,activity,name,sex_,age,injury,fatal_(y/n),time,species_,investigator_or_source,pdf,href_formula,href,case_number.1,case_number.2,original_order,unnamed:_22,unnamed:_23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adysonï¿½McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25718,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25719,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25720,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25721,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**CLEANING**

First we we need to drop all the rows with null values.

In [3]:
attacks_subset = shark_attacks.dropna(how='all')
attacks_subset.isna().sum()

case_number                  1
date                      2401
year                      2403
type                      2405
country                   2451
area                      2856
location                  2941
activity                  2945
name                      2611
sex_                      2966
age                       5232
injury                    2429
fatal_(y/n)               2940
time                      5755
species_                  5239
investigator_or_source    2418
pdf                       2401
href_formula              2402
href                      2401
case_number.1             2401
case_number.2             2401
original_order            2394
unnamed:_22               8702
unnamed:_23               8701
dtype: int64

Now we need to have a look to those rows were the 'case_number' is not null but the rest of the values are null.

In [4]:
rows_with_nan = attacks_subset[attacks_subset.drop('case_number', axis=1).isna().all(axis=1)]
print(rows_with_nan.shape)
rows_with_nan.isna().sum()

(2394, 24)


case_number                  0
date                      2394
year                      2394
type                      2394
country                   2394
area                      2394
location                  2394
activity                  2394
name                      2394
sex_                      2394
age                       2394
injury                    2394
fatal_(y/n)               2394
time                      2394
species_                  2394
investigator_or_source    2394
pdf                       2394
href_formula              2394
href                      2394
case_number.1             2394
case_number.2             2394
original_order            2394
unnamed:_22               2394
unnamed:_23               2394
dtype: int64

As we can see, all the columns unless 'case_number' have the same null values as the lenght of the rows filtered, then we can remove this rows because they do not have any information.

In [5]:
# The '~' operator is used to negate the boolean condition, so it keeps the rows where the condition is False.
attacks_subset_ = attacks_subset[~attacks_subset.drop('case_number', axis=1).isna().all(axis=1)]
print(attacks_subset_.shape)
attacks_subset_[attacks_subset_['case_number'] == '0']

(6309, 24)


,case_number,date,year,type,country,area,location,activity,name,sex_,age,injury,fatal_(y/n),time,species_,investigator_or_source,pdf,href_formula,href,case_number.1,case_number.2,original_order,unnamed:_22,unnamed:_23
6302,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6304.0,NaN,NaN
6303,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6305.0,NaN,NaN
6304,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6306.0,NaN,NaN
6305,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6307.0,NaN,NaN
6306,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6308.0,NaN,NaN
6307,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6309.0,NaN,NaN
6308,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6310.0,NaN,NaN


We still have some rows with almost all the data with null values so as I don't need the column 'original_order', I can remove this rows.

In [6]:
attacks_subset = attacks_subset[~attacks_subset.drop(['case_number', 'original_order'], axis=1).isna().all(axis=1)]
print(attacks_subset.shape)

(6302, 24)


Now I want to remove those columns that are not giving me any type of information.

In [7]:
attacks_subset = attacks_subset.drop(columns=['pdf', 'href_formula', 'href', 'case_number.1', 'case_number.2', 'original_order', 'unnamed:_22', 'unnamed:_23'])
attacks_subset


,case_number,date,year,type,country,area,location,activity,name,sex_,age,injury,fatal_(y/n),time,species_,investigator_or_source
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF"
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adysonï¿½McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com"
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com"
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF"
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6297,ND.0005,Before 1903,0.0,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,male,M,NaN,FATAL,Y,NaN,NaN,"H. Taunton; N. Bartlett, p. 234"
6298,ND.0004,Before 1903,0.0,Unprovoked,AUSTRALIA,Western Australia,NaN,Pearl diving,Ahmun,M,NaN,FATAL,Y,NaN,NaN,"H. Taunton; N. Bartlett, pp. 233-234"
6299,ND.0003,1900-1905,0.0,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,Coast Guard personnel,M,NaN,FATAL,Y,NaN,NaN,"F. Schwartz, p.23; C. Creswell, GSAF"
6300,ND.0002,1883-1889,0.0,Unprovoked,PANAMA,NaN,"Panama Bay 8ï¿½N, 79ï¿½W",NaN,Jules Patterson,M,NaN,FATAL,Y,NaN,NaN,"The Sun, 10/20/1938"


In order to find the duplicated rows we do it by 'original_order'.

In [8]:
duplicated = attacks_subset[attacks_subset['case_number'].duplicated(keep=False)]
duplicated

,case_number,date,year,type,country,area,location,activity,name,sex_,age,injury,fatal_(y/n),time,species_,investigator_or_source
522,2014.08.02,02-Aug-2014,2014.0,Unprovoked,USA,Florida,"South of Cocoa Beach, Brevard County",Surfing,male,M,50s,Foot bitten,N,NaN,NaN,"Florida Today, 8/8/2014"
523,2014.08.02,02-Aug-2014,2014.0,Unprovoked,USA,Florida,"Table Beach, Brevard County",Boogie boarding,Christian Sanhueza,M,8,Laceration to ankle,N,13h00,NaN,"Florida Today, 8/2/2014"
615,2013.10.05,06-Oct-2013,2013.0,Unprovoked,USA,California,"Bunkers, Humboldt Bay, Eureka, Humboldt County",Surfing,Jay Scrivner,M,45,Laceration to thigh,N,08h45,"White shark, 8' to 10'","R. Collier, GSAF"
616,2013.10.05,10-Oct-2013,2013.0,Unprovoked,USA,Florida,"Destin, Okaloosa County",Wading,Zachary Tyke Standridge,M,12,Lacerations to right forearm,N,15h30,Small bull shark,"Monroe County Advocate, 10/9/2013"
746,2012.09.02.b,02-Sep-2012,2012.0,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Boogie boarding,female,F,8,Puncture wounds to calf and hand,N,18h30,3.5' to 4' shark,"WYTV, 9/3/2012"
747,2012.09.02.b,02-Sep-2012,2012.0,Provoked,USA,Hawaii,"Spreckelsville, Maui",Spearfishing,M. Malabon,NaN,NaN,Minor laceration to hand PROVOKED INCIDENT,N,12h00,"Tiger shark, 10' to 12'",HawaiiNow.com
1063,2009.12.18,18-Dec-2009,2009.0,Unprovoked,SOUTH AFRICA,Eastern Cape Province,"Second Beach, Port St. Johns",Paddling on kneeboard,Tshintshekile Nduva,M,22,FATAL,Y,14h30,NaN,"B. Jordan & A. Ferreira, Times Live, 12/21/2009"
1064,2009.12.18,18-Dec-2009,2009.0,Invalid,SOUTH AFRICA,KwaZulu-Natal,"North Beach, Durban",Surfing,Lance Morris,M,NaN,Minor lacerations to left leg. nitially report...,NaN,NaN,No shark involvement,"M. Addison, C. Eckstander, GSAF"
1436,2006.09.02,02-Sep-2006,2006.0,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Swimming,male,M,12 or 13,Arm bitten,N,17h55,NaN,"S. Petersohn, GSAF"
1437,2006.09.02,02-Sep-2006,2006.0,Unprovoked,SOUTH AFRICA,Western Cape Province,Noordhoek,Surfing,Steven Harcourt-Wood,M,37,"No injury, shark rammed surfboard",N,NaN,"White shark, 3.5m","Cape Times, 9/3/2006"


We check that there are no rows duplicated with the same information.

So now we can start cleaning the columns in order to standarize the data.

    1. Let's start with 'type' column of shark attack

In [9]:
attacks_subset['type'].value_counts()

type
Unprovoked      4595
Provoked         574
Invalid          547
Sea Disaster     239
Boating          203
Boat             137
Questionable       2
Boatomg            1
Name: count, dtype: int64

In [10]:
pattern = r'\w*[Bb]oat\w*'
attacks_subset['type'] = attacks_subset['type'].str.replace(pattern, 'Boat', regex=True)
attacks_subset['type'].value_counts()

type
Unprovoked      4595
Provoked         574
Invalid          547
Boat             341
Sea Disaster     239
Questionable       2
Name: count, dtype: int64

    2. For 'time' column I want to classify hours by ranges of time.

In [11]:
attacks_subset['time'].unique()

array(['18h00', '14h00  -15h00', '07h45', nan, 'Late afternoon', '17h00',
       '14h00', 'Morning', '15h00', '08h15', '11h00', '10h30', '10h40',
       '16h50', '07h00', '09h30', 'Afternoon', '21h50', '09h40', '08h00',
       '17h35', '15h30', '07h30', '19h00, Dusk', 'Night', '16h00',
       '15h01', '12h00', '13h45', '23h30', '09h00', '14h30', '18h30',
       '12h30', '16h30', '18h45', '06h00', '10h00', '10h44', '13h19',
       'Midday', '13h30', '10h45', '11h20', '11h45', '19h30', '08h30',
       '15h45', 'Shortly before 12h00', '17h34', '17h10', '11h15',
       '08h50', '17h45', '13h00', '10h20', '13h20', '02h00', '09h50',
       '11h30', '17h30', '9h00', '10h43', 'After noon', '15h15', '15h40',
       '19h05', '1300', '14h30 / 15h30', '22h00', '16h20', '14h34',
       '15h25', '14h55', '17h46', 'Morning ', '15h49', '19h00',
       'Midnight', '09h30 / 10h00', '10h15', '18h15', '04h00', '14h50',
       '13h50', '19h20', '10h25', '10h45-11h15', '16h45', '15h52',
       '06h15', '14h

In [12]:
hour_pattern = r'\d{1,2}(?::|h|j)\d{2}'
second_patter = r'\d{1,2}\w\d{2}|>\d{1,2}:\d{2}|(0?\d|1[0-2]):\d{2}j|Before (0?\d|1[0-2]):\d{2}|Between (0?\d|1[0-2]):\d{2} and (0?\d|1[0-2]):\d{2}|\d{1,2}:\d{2}|\d{1,2}:\d{2}'

def formatt_hour(string):
    if 'h' in string:
        string = string.split('h')
        if len(string[0]) == 1:
            string[0] += '0'
        return ":".join(string)
    elif 'j' in string:
        return string.replace('j', ':')
    elif string.isdigit():
        string = string[:len(string)//2] + ':' + string[len(string)//2:]
    else:
        return string



In [13]:
def class_by_string(string):
    if 'morning' in str(string).lower():
        return 'Morning'
    elif 'afternoon' in str(string).lower() or 'midday' in str(string).lower() or 'noon' in str(string).lower():
        return "Afternoon"
    elif 'evening' in str(string).lower():
        return "Evening"
    elif 'night' in str(string).lower() or 'dusk' in str(string).lower():
        return "Night"
    elif re.search(second_patter, str(string)):
        str_ = re.search(second_patter, str(string))
        return formatt_hour(str_.group())
    else:
        return "Unknown"

attacks_subset['clean_time'] = attacks_subset['time'].apply(class_by_string)
attacks_subset

,case_number,date,year,type,country,area,location,activity,name,sex_,age,injury,fatal_(y/n),time,species_,investigator_or_source,clean_time
0,2018.06.25,25-Jun-2018,2018.0,Boat,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",18:00
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adysonï¿½McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",14:00
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",07:45
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",Unknown
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,Unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6297,ND.0005,Before 1903,0.0,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,male,M,NaN,FATAL,Y,NaN,NaN,"H. Taunton; N. Bartlett, p. 234",Unknown
6298,ND.0004,Before 1903,0.0,Unprovoked,AUSTRALIA,Western Australia,NaN,Pearl diving,Ahmun,M,NaN,FATAL,Y,NaN,NaN,"H. Taunton; N. Bartlett, pp. 233-234",Unknown
6299,ND.0003,1900-1905,0.0,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,Coast Guard personnel,M,NaN,FATAL,Y,NaN,NaN,"F. Schwartz, p.23; C. Creswell, GSAF",Unknown
6300,ND.0002,1883-1889,0.0,Unprovoked,PANAMA,NaN,"Panama Bay 8ï¿½N, 79ï¿½W",NaN,Jules Patterson,M,NaN,FATAL,Y,NaN,NaN,"The Sun, 10/20/1938",Unknown


In [14]:
#regex_condition = attacks_subset['day_time'].str.contains(hour_pattern)
# Aplica la función 'formatt_hour' solo a los valores que cumplen el patrón regex
#non_null_condition = (regex_condition) & (~attacks_subset['day_time'].isnull())
#attacks_subset.loc[non_null_condition, 'day_time'] = attacks_subset.loc[non_null_condition, 'day_time'].apply(formatt_hour)
#attacks_subset['day_time'].unique()

In [15]:
def class_by_time(string):
    """
    This function receives a string in the format '12:00' and determines whether this time is 'Morning', 'Afternoon', 'Evening' or 'Night'.
    """
    morning_pattern = r'0[6-9]:[0-5][0-9]|1[0-1]:[0-5][0-9]'
    afternoon_pattern = r'1[2-7]:[0-5][0-9]'
    evening_pattern = r'1[8-9]:[0-5][0-9]|20:[0-5][0-9]|21:[0-5][0-9]'
    night_pattern = r'22:[0-5][0-9]|23:[0-5][0-9]|0[0-5]:[0-5][0-9]'

    if re.search(morning_pattern, str(string)):
        return 'Morning'
    elif re.search(afternoon_pattern, str(string)):
        return "Afternoon"
    elif re.search(evening_pattern, str(string)):
        return "Evening"
    elif re.search(night_pattern, str(string)):
        return "Night"
    else:
        return 'Unknown'
    
pattern = r'\d{1,2}:\d{1,2}'
attacks_subset['clean_time'] = attacks_subset['clean_time'].apply(lambda x: class_by_time(x) if pd.notna(x) and pd.Series(x).str.contains(pattern).any() else x)

attacks_subset['clean_time'].value_counts()




clean_time
Unknown      3414
Afternoon    1566
Morning       914
Evening       254
Night         146
Name: count, dtype: int64

- The cleaning of time column is useless because more than half of the data is 'Unknown'.

    3. Now let's clean 'age' column.

In [16]:
attacks_subset['age'].unique()

array(['57', '11', '48', nan, '18', '52', '15', '12', '32', '10', '21',
       '34', '30', '60', '33', '29', '54', '41', '37', '56', '19', '25',
       '69', '38', '55', '35', '46', '45', '14', '40s', '28', '20', '24',
       '26', '49', '22', '7', '31', '17', '40', '13', '42', '3', '8',
       '50', '16', '82', '73', '20s', '68', '51', '39', '58', 'Teen',
       '47', '61', '65', '36', '66', '43', '60s', '9', '72', '59', '6',
       '27', '64', '23', '71', '44', '62', '63', '70', '18 months', '53',
       '30s', '50s', 'teen', '77', '74', '28 & 26', '5', '86', '18 or 20',
       '12 or 13', '46 & 34', '28, 23 & 30', 'Teens', '36 & 26',
       '8 or 10', '84', 'ï¿½ ', ' ', '30 or 36', '6ï¿½', '21 & ?', '75',
       '33 or 37', 'mid-30s', '23 & 20', ' 30', '7      &    31', ' 28',
       '20?', "60's", '32 & 30', '16 to 18', '87', '67', 'Elderly',
       'mid-20s', 'Ca. 33', '74 ', '45 ', '21 or 26', '20 ', '>50',
       '18 to 22', 'adult', '9 & 12', '? & 19', '9 months', '25 to 35',
 

In [17]:
def extract_age(x):
    if pd.isna(x):
        return 'Unknown'
    if x in ['teen', 'Teen', 'Teens']:
        return '15'
    if x in ['adult', '(adult)', 'middle-aged']:
        return '50'
    if 'months' in x:
        print(x)
        x = x.split(' ')[0]
        if int(x) < 12:
            return '< 1'
        else:
            return '2'
    age_av = re.findall(r'(\d{1,2})\s*(&|or|to)\s*(\d{1,2})', str(x))
    if age_av:
        average_ages = [(int(match[0]) + int(match[2])) // 2 for match in age_av]
        return str(average_ages[0])
    age_match = re.search(r'\d{1,2}', str(x))
    if age_match:
        return age_match.group()
    return 'Unknown'

In [18]:
attacks_subset['clean_age'] = attacks_subset['age'].apply(extract_age)
attacks_subset['clean_age'].unique()

18 months
9 months
2 to 3 months


array(['57', '11', '48', 'Unknown', '18', '52', '15', '12', '32', '10',
       '21', '34', '30', '60', '33', '29', '54', '41', '37', '56', '19',
       '25', '69', '38', '55', '35', '46', '45', '14', '40', '28', '20',
       '24', '26', '49', '22', '7', '31', '17', '13', '42', '3', '8',
       '50', '16', '82', '73', '68', '51', '39', '58', '47', '61', '65',
       '36', '66', '43', '9', '72', '59', '6', '27', '64', '23', '71',
       '44', '62', '63', '70', '2', '53', '77', '74', '5', '86', '84',
       '75', '87', '67', '< 1', '1', '81', '78'], dtype=object)

In [19]:
attacks_subset

,case_number,date,year,type,country,area,location,activity,name,sex_,age,injury,fatal_(y/n),time,species_,investigator_or_source,clean_time,clean_age
0,2018.06.25,25-Jun-2018,2018.0,Boat,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",Evening,57
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adysonï¿½McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",Afternoon,11
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",Morning,48
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",Unknown,Unknown
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,Unknown,Unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6297,ND.0005,Before 1903,0.0,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,male,M,NaN,FATAL,Y,NaN,NaN,"H. Taunton; N. Bartlett, p. 234",Unknown,Unknown
6298,ND.0004,Before 1903,0.0,Unprovoked,AUSTRALIA,Western Australia,NaN,Pearl diving,Ahmun,M,NaN,FATAL,Y,NaN,NaN,"H. Taunton; N. Bartlett, pp. 233-234",Unknown,Unknown
6299,ND.0003,1900-1905,0.0,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,Coast Guard personnel,M,NaN,FATAL,Y,NaN,NaN,"F. Schwartz, p.23; C. Creswell, GSAF",Unknown,Unknown
6300,ND.0002,1883-1889,0.0,Unprovoked,PANAMA,NaN,"Panama Bay 8ï¿½N, 79ï¿½W",NaN,Jules Patterson,M,NaN,FATAL,Y,NaN,NaN,"The Sun, 10/20/1938",Unknown,Unknown


    4. Column 'year'

In [20]:
attacks_subset['year'].unique()

array([2018., 2017.,   nan, 2016., 2015., 2014., 2013., 2012., 2011.,
       2010., 2009., 2008., 2007., 2006., 2005., 2004., 2003., 2002.,
       2001., 2000., 1999., 1998., 1997., 1996., 1995., 1984., 1994.,
       1993., 1992., 1991., 1990., 1989., 1969., 1988., 1987., 1986.,
       1985., 1983., 1982., 1981., 1980., 1979., 1978., 1977., 1976.,
       1975., 1974., 1973., 1972., 1971., 1970., 1968., 1967., 1966.,
       1965., 1964., 1963., 1962., 1961., 1960., 1959., 1958., 1957.,
       1956., 1955., 1954., 1953., 1952., 1951., 1950., 1949., 1948.,
       1848., 1947., 1946., 1945., 1944., 1943., 1942., 1941., 1940.,
       1939., 1938., 1937., 1936., 1935., 1934., 1933., 1932., 1931.,
       1930., 1929., 1928., 1927., 1926., 1925., 1924., 1923., 1922.,
       1921., 1920., 1919., 1918., 1917., 1916., 1915., 1914., 1913.,
       1912., 1911., 1910., 1909., 1908., 1907., 1906., 1905., 1904.,
       1903., 1902., 1901., 1900., 1899., 1898., 1897., 1896., 1895.,
       1894., 1893.,

In [21]:
pattern_year = r'\d{4}'
str_pattern = r'\d{4}\.'
def clean_year(year):
    if re.match(str_pattern, str(year)):
        return re.match(pattern_year, str(year)).group()
    else:
        return 'Unknown'

attacks_subset['clean_year'] = attacks_subset['year'].apply(clean_year)
attacks_subset['clean_year'].unique()
attacks_subset.sample(10)


,case_number,date,year,type,country,area,location,activity,name,sex_,age,injury,fatal_(y/n),time,species_,investigator_or_source,clean_time,clean_age,clean_year
2549,1992.06.19,19-Jun-1992,1992.0,Unprovoked,VANUATU,Malampa Province,"Port Sandwich, Malakula",Swimming,Andrea Rush,F,21,Right leg was bitten above and below the knee ...,N,NaN,NaN,"A. Rush; R. D. Weeks, GSAF; Otago Daily Times,...",Unknown,21,1992
2502,1993.03.17,17-Mar-1993,1993.0,Unprovoked,JAPAN,Ehime Prefecture,Iyo,NaN,Yuji Yamamoto,NaN,NaN,Survived,N,NaN,Possiby white shark,K. Nakaya,Unknown,Unknown,1993
2296,1996.07.10,10-Jul-1996,1996.0,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",NaN,Andrew Lewis,M,18,Right foot bitten,N,13h45,NaN,"S. Petersohn, GSAF",Afternoon,18,1996
1672,2004.05.22.a,22-May-2004,2004.0,Unprovoked,BRAZIL,Pernambuco,"Piedade Beach, Recife",Wading,Naiane Barbosa Bringel,F,24,Hips & thighs bitten,N,16h00,NaN,"P.M. Lopes, GSAF, JC, 5/23/2004",Afternoon,24,2004
1949,2001.05.04,04-May-2001,2001.0,Unprovoked,AUSTRALIA,New South Wales,Noraville,Surfing,Michael Valentine,M,33 or 37,"Chest lacerated, surfboard bitten",N,17h45,NaN,"Newcastle Herald; Sunday Mail (QLD), 5/6/2001,...",Afternoon,35,2001
590,2013.12.25,25-Dec-2013,2013.0,Unprovoked,NEW CALEDONIA,North Province,"Lindï¿½ralique, Hienghï¿½ne",Snorkeling,Loï¿½c Merlet,M,37,Leg bitten,N,11h00,Reported to involve a bull shark,"Les Novelles Caledonie, 12/26/2014",Morning,37,2013
533,2014.07.12,12-Jul-2014,2014.0,Unprovoked,USA,North Carolina,"Masonboro Island, New Hanover County",Body surfing,Carson Jewell,M,NaN,Lacerations to hand and wrist,N,NaN,NaN,"C. Creswell, GSAF",Unknown,Unknown,2014
1480,2006.05.21,21-May-2006,2006.0,Unprovoked,BRAZIL,Pernambuco,"Boa Viagem Beach, Recife",Surfing,Rogï¿½rio Antï¿½nio de Carvalho,M,33,"Injuries to left thigh, calf & foot",N,11h30,NaN,"Avisa Nordland, 5/22/2006",Morning,33,2006
1725,2003.10.31.c,31-Oct-2003,2003.0,Unprovoked,USA,Florida,"Ponte Vedra Beach, St. Johns County",Surfing,Adam Gray,M,18,Left foot (sole) bitten,N,15h00,1.2 m to 1.5 m [4' to 5'] shark,"D. Dixon, Florida Times-Union, 11/8/2003",Afternoon,18,2003
5242,1921.09.00,Sep-1921,1921.0,Provoked,ENGLAND,Dorset,Weymouth,Fishing,Roberts,M,NaN,Leg bitten by shark he was attempting to captu...,N,NaN,"Blue shark, 4'","Argus, 10/8/1921",Unknown,Unknown,1921


In [29]:
attacks_subset['clean_year'].unique()

array(['2018', '2017', 'Unknown', '2016', '2015', '2014', '2013', '2012',
       '2011', '2010', '2009', '2008', '2007', '2006', '2005', '2004',
       '2003', '2002', '2001', '2000', '1999', '1998', '1997', '1996',
       '1995', '1984', '1994', '1993', '1992', '1991', '1990', '1989',
       '1969', '1988', '1987', '1986', '1985', '1983', '1982', '1981',
       '1980', '1979', '1978', '1977', '1976', '1975', '1974', '1973',
       '1972', '1971', '1970', '1968', '1967', '1966', '1965', '1964',
       '1963', '1962', '1961', '1960', '1959', '1958', '1957', '1956',
       '1955', '1954', '1953', '1952', '1951', '1950', '1949', '1948',
       '1848', '1947', '1946', '1945', '1944', '1943', '1942', '1941',
       '1940', '1939', '1938', '1937', '1936', '1935', '1934', '1933',
       '1932', '1931', '1930', '1929', '1928', '1927', '1926', '1925',
       '1924', '1923', '1922', '1921', '1920', '1919', '1918', '1917',
       '1916', '1915', '1914', '1913', '1912', '1911', '1910', '1909',
   

    5. Column 'sex'

In [22]:
attacks_subset['sex_'].unique()

array(['F', 'M', nan, 'M ', 'lli', 'N', '.'], dtype=object)

In [23]:
def sex_clean(x):
    """
    This function standarizes all the values found in the dataframe sex column.
    Params:
        x (str): Value of the sex
    Returns:
        string (str): Standarized sex
    """
    if x != 'M' and x != 'F' and x != 'M ':
        return 'Unknown'
    elif x == 'M ':
        return 'M'
    else:
        return x

In [24]:
attacks_subset['clean_sex'] = attacks_subset['sex_'].apply(sex_clean)
attacks_subset['clean_sex'].unique()

array(['F', 'M', 'Unknown'], dtype=object)

    6. Column 'activity'

In [25]:
set(attacks_subset['activity'])


{'Swimming, after boat swamped',
 'Grabbed shark & threw it on deck',
 'Free diving but treading water at surface',
 'Bathing or body surfing ',
 'Leaving the water',
 'Wading or swimming',
 'Crossing river on a raft',
 'Dropping anchor',
 'Killing a shark',
 'Fishing with dynamite',
 'Leicester abandoned in a hurricane',
 'Standing, washing rear wheels of his ambulance in ankle-deep water',
 "Attempting to lasso shark's tail",
 'Fishing, tossing netted shark onboard',
 'A junk foundered',
 'Parachuted from balloon',
 'Sinking of the M/V Mindoro during a typhoon',
 'Walking in chest-deep water',
 'Standing',
 'Playing in surf with his child  (9)',
 'Floating',
 'Fishing from boat, Kaimamla',
 'Grabbing shark for a selfie',
 'Plane crashed in water, men in life raft',
 '2 boats capsized',
 'Pearl diving, but standing in the water',
 'Swimming at edge of channel',
 'Taking boat from California to Florida when it ran aground & he was swimming back to boat',
 'A dhow capsized',
 'Skindivin

In [26]:
def replace_activities(activity):
    activities = {
        'Sailing': ['boat', 'boating', 'racing', 'barqued', 'sinking', 'ship', 'wreck', 'dhow', 'kayak', 'canoa', 'raft', 'cutter', 'bark', 'submarine'],
        'Fishing': ['fishing', 'fish', 'spearfishing', 'netting', 'wade-fishing', 'hunting', 'fishingat', 'shrimping', 'gigging', 'picking'],
        'Air accident': ['aircraft', 'airliner', 'constellation', 'parachute'],
        'Swimming': ['swimming', 'riding'],
        'Diving': ['diving', 'diver', 'photographing', 'dive', 'skidiving'],
        'Surfing': ['surfing', 'surf', 'boogie', 'surf-skiing', 'paddeling', 'boarding', 'board', 'overboard', 'treading'],
        'Dangerous': ['explosives', 'anesthesize', 'investigating', 'pulling', 'filming', 'defecating', 'chumming', 'wrangling', 'murder', 'searching'],
        'Bathing': ['playing', 'bath', 'bathing', 'crouching', 'floating', 'standing', 'sitting', 'dangling'],
        'Natural disaster': ['hurricane', 'natural']
    }

    for key, word_list in activities.items():
        if any(keyword in str(activity).lower() for keyword in word_list):
            return key
    return 'Unknown'

attacks_subset['clean_activity'] = attacks_subset['activity'].apply(replace_activities)
attacks_subset['clean_activity'].value_counts()

clean_activity
Surfing             1465
Unknown             1237
Fishing             1174
Swimming            1067
Diving               528
Sailing              417
Bathing              379
Dangerous             26
Air accident           5
Natural disaster       4
Name: count, dtype: int64

In [27]:
attacks_subset

,case_number,date,year,type,country,area,location,activity,name,sex_,age,injury,fatal_(y/n),time,species_,investigator_or_source,clean_time,clean_age,clean_year,clean_sex,clean_activity
0,2018.06.25,25-Jun-2018,2018.0,Boat,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",Evening,57,2018,F,Unknown
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adysonï¿½McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",Afternoon,11,2018,F,Bathing
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",Morning,48,2018,M,Surfing
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",Unknown,Unknown,2018,M,Surfing
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,Unknown,Unknown,2018,M,Diving
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6297,ND.0005,Before 1903,0.0,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,male,M,NaN,FATAL,Y,NaN,NaN,"H. Taunton; N. Bartlett, p. 234",Unknown,Unknown,Unknown,M,Diving
6298,ND.0004,Before 1903,0.0,Unprovoked,AUSTRALIA,Western Australia,NaN,Pearl diving,Ahmun,M,NaN,FATAL,Y,NaN,NaN,"H. Taunton; N. Bartlett, pp. 233-234",Unknown,Unknown,Unknown,M,Diving
6299,ND.0003,1900-1905,0.0,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,Coast Guard personnel,M,NaN,FATAL,Y,NaN,NaN,"F. Schwartz, p.23; C. Creswell, GSAF",Unknown,Unknown,Unknown,M,Swimming
6300,ND.0002,1883-1889,0.0,Unprovoked,PANAMA,NaN,"Panama Bay 8ï¿½N, 79ï¿½W",NaN,Jules Patterson,M,NaN,FATAL,Y,NaN,NaN,"The Sun, 10/20/1938",Unknown,Unknown,Unknown,M,Unknown


**Let's export the clean dataframe to csv**

In [28]:
attacks_subset.to_csv('../data/clean_attacks.csv', index=False)